Encode with the scaled data


In [ ]:
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl

In [1]:
import torch
import numpy as np
import torch.nn as nn
from sklearn.decomposition import PCA

# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_dataset = torch.load('/content/drive/MyDrive/Autoencoding/raw_cv_whiten.pt')

In [ ]:
def make_4_dim(data):
    data=torch.unsqueeze(data,1)
    data = torch.unsqueeze(data,3)
    return data

def make_2_dim(data):
    data=torch.squeeze(data,1)
    data = torch.squeeze(data,2)
    return data
def conv_block1(in_channels,out_channels,kernel_size,stride,padding):
    layers = [nn.ConvTranspose2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,
                                 stride=stride,padding=padding),
              nn.BatchNorm2d(out_channels),
              nn.LeakyReLU(negative_slope=0.2,inplace=True)]
    return nn.Sequential(*layers)

def conv_block2(in_channels,out_channels,kernel_size,stride,padding,pool=False):
    layers = [nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,
                        stride=stride,padding=padding),
              nn.BatchNorm2d(out_channels),
              nn.LeakyReLU(negative_slope=0.2,inplace=True)]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)
    

class Resnet9(nn.Module):
      def __init__(self, in_channel, out_channel):
        super().__init__()
        
        # Encode1
        
        self.conv1 = conv_block1(in_channel, 64,kernel_size=4,stride=2,padding=0)         ## 64*6*5
        self.conv2 = conv_block1(64, 128,kernel_size=4,stride=2,padding=0)                ## 128*14*10
        self.conv3 = conv_block1(128, 256,kernel_size=4,stride=2,padding=0)               ##256*30*22
        self.res1 = nn.Sequential(conv_block1(256, 256,kernel_size=3,stride=1,padding=1), ## 256*30*22  
                                  conv_block1(256, 256,kernel_size=3,stride=1,padding=1))  ##256*30*22
        
        #Encode2
        self.conv4 = conv_block2(256, 128,kernel_size=4,stride=2,padding=0)         ## 128*14*10
        self.conv5 = conv_block2(128, 64,kernel_size=4,stride=2,padding=0)                ## 64*6*5
        #self.conv6 = conv_block2(128, 256,kernel_size=4,stride=2,padding=0)               ##256*30*22
        self.res2 = nn.Sequential(conv_block2(64,64,kernel_size=3,stride=1,padding=1), ## 256*30*22  
                                  conv_block2(64,64,kernel_size=3,stride=1,padding=1))  ##256*30*22
        self.conv6 = conv_block2(64,1,kernel_size=4,stride=2,padding=0)                 #1*2*1
        
        #Decode 1
        self.conv7 = conv_block1(in_channel, 64,kernel_size=4,stride=2,padding=0)         ## 64*6*5
        self.conv8 = conv_block1(64, 128,kernel_size=4,stride=2,padding=0)                ## 128*14*10
        self.conv9 = conv_block1(128, 256,kernel_size=4,stride=2,padding=0)               ##256*30*22
        self.res3 = nn.Sequential(conv_block1(256, 256,kernel_size=3,stride=1,padding=1), ## 256*30*22  
                                  conv_block1(256, 256,kernel_size=3,stride=1,padding=1))  ##256*30*22
        
        #Decode 2
        self.conv10 = conv_block2(256, 128,kernel_size=4,stride=2,padding=0)         ## 128*14*10
        self.conv11 = conv_block2(128, 64,kernel_size=4,stride=2,padding=0)                ## 64*6*5
        #self.conv6 = conv_block2(128, 256,kernel_size=4,stride=2,padding=0)               ##256*30*22
        self.res4 = nn.Sequential(conv_block2(64,64,kernel_size=3,stride=1,padding=1), ## 256*30*22  
                                  conv_block2(64,64,kernel_size=3,stride=1,padding=1))  ##256*30*22
        self.conv12 = nn.Conv2d(64,out_channel,kernel_size=4,stride=2,padding=0)                 #1*2*1
        
      def encode(self,in_data):
          out = self.conv1(in_data.float())
          out = self.conv2(out)
          out = self.conv3(out)
          out = self.res1(out)+out
          out = self.conv4(out)
          out = self.conv5(out)
          out = self.res2(out)+out
          out = self.conv6(out)
          return out
           
      def decode(self,lat_data):
          out = self.conv7(lat_data.float())
          out = self.conv8(out)
          out = self.conv9(out)
          out = self.res3(out)+out
          out = self.conv10(out)
          out = self.conv11(out)
          out = self.res4(out)+out
          out = self.conv12(out)
          return out

In [ ]:
device=torch.device('cuda')
encoder = Resnet9(1,1).to(device)

In [ ]:
encoder.load_state_dict(torch.load('/content/drive/MyDrive/TAE_DC/encoder_state_dict_resnet.pth'))

<All keys matched successfully>

In [ ]:
#data = train_dataset[:]

In [ ]:
x= int(4022018/2002)
print(x)

2009


In [ ]:
import gc
gc.collect()

152

In [ ]:
from tqdm.notebook  import tqdm

In [ ]:
who_ls

['PCA',
 'Resnet9',
 'conv_block1',
 'conv_block2',
 'device',
 'drive',
 'encoder',
 'gc',
 'make_2_dim',
 'make_4_dim',
 'nn',
 'np',
 'torch',
 'tqdm',
 'train_dataset',
 'x']

In [ ]:
#lat1 = make_2_dim(encoder.encode(make_4_dim(train_dataset.to(device))))

In [ ]:
%%time
#latent = []
for i in tqdm(range(1,2003)):
    lat1 = make_2_dim(encoder.encode(make_4_dim(train_dataset[(i-1)*x:(i*x)].to(device))))
    torch.save(lat1,'lat'+str(i)+'.pt')
    #latent.append(lat1)
    del lat1 
    gc.collect()


CPU times: user 8min 48s, sys: 5min 30s, total: 14min 18s
Wall time: 14min 16s


In [ ]:
import glob

In [ ]:
file = sorted(glob.glob('lat*'))

In [ ]:
file

['lat1.pt',
 'lat10.pt',
 'lat100.pt',
 'lat1000.pt',
 'lat1001.pt',
 'lat1002.pt',
 'lat1003.pt',
 'lat1004.pt',
 'lat1005.pt',
 'lat1006.pt',
 'lat1007.pt',
 'lat1008.pt',
 'lat1009.pt',
 'lat101.pt',
 'lat1010.pt',
 'lat1011.pt',
 'lat1012.pt',
 'lat1013.pt',
 'lat1014.pt',
 'lat1015.pt',
 'lat1016.pt',
 'lat1017.pt',
 'lat1018.pt',
 'lat1019.pt',
 'lat102.pt',
 'lat1020.pt',
 'lat1021.pt',
 'lat1022.pt',
 'lat1023.pt',
 'lat1024.pt',
 'lat1025.pt',
 'lat1026.pt',
 'lat1027.pt',
 'lat1028.pt',
 'lat1029.pt',
 'lat103.pt',
 'lat1030.pt',
 'lat1031.pt',
 'lat1032.pt',
 'lat1033.pt',
 'lat1034.pt',
 'lat1035.pt',
 'lat1036.pt',
 'lat1037.pt',
 'lat1038.pt',
 'lat1039.pt',
 'lat104.pt',
 'lat1040.pt',
 'lat1041.pt',
 'lat1042.pt',
 'lat1043.pt',
 'lat1044.pt',
 'lat1045.pt',
 'lat1046.pt',
 'lat1047.pt',
 'lat1048.pt',
 'lat1049.pt',
 'lat105.pt',
 'lat1050.pt',
 'lat1051.pt',
 'lat1052.pt',
 'lat1053.pt',
 'lat1054.pt',
 'lat1055.pt',
 'lat1056.pt',
 'lat1057.pt',
 'lat1058.pt',
 'lat1

In [ ]:
X = []
for f in file:
    X.append(torch.load(f))

X1 = torch.tensor(torch.cat(X))


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """


In [ ]:
rm -r lat*

In [ ]:
torch.save(X1,'latent.pt')

In [ ]:
X1.size()

torch.Size([4022018, 2])